In [3]:
import dspy

gpt41mini = dspy.LM('azure/gpt-4.1-mini', temperature=0.7)
gpt41 = dspy.LM('azure/gpt-4.1', temperature=0.7)

dspy.configure(lm=gpt41mini)

In [4]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

Generating test split: 100%|██████████| 4000/4000 [00:00<00:00, 51961.63 examples/s]


In [5]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Chris Noonan', 'Miss Potter', 'Academy Award for Best Director']


In [6]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [7]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [8]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [9]:
react(claim="David Gregory was born in 1625.").titles[:3]

['David Gregory (physician)']

In [10]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [11]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 86.33 / 100 (86.3%): 100%|██████████| 100/100 [02:42<00:00,  1.63s/it]

2025/05/01 14:04:39 INFO dspy.evaluate.evaluate: Average Metric: 86.33333333333333 / 100 (86.3%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...","{'thought_0': ""To verify this claim, I need to identify the Church...",The claim states that the Church of England's movement inspiring t...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",✔️ [1.000]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]","{'thought_0': 'The claim references ""Red, White & Crüe"" and a Fren...","The claim connects ""Red, White & Crüe,"" which is identified as an ...","[Red, White & Crüe, Bobby Stewart, Mike Tyson, Christophe Mendy, F...",✔️ [0.667]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Ed Wood, Glen or Glenda, Fernand Rivers]","{'thought_0': 'To verify the claim, I need to first identify the w...","The claim states that the writer/director/actor from ""Glen or Glen...","[Glen or Glenda, Ed Wood, Fernand Rivers]",✔️ [1.000]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[The End of Suburbia, Sandi Sissel, Chicken Ranch (film)]","{'thought_0': 'To verify the claim, I need to find the film direct...","The claim states that a film by Sandi Sissel was released before ""...","[Sandi Sissel, The End of Suburbia, Chicken Ranch (film)]",✔️ [1.000]
4,The actor who played captain hook in the live production with Tayl...,"[Christopher Walken, Peter Pan Live!, Taylor Louderman]","{'thought_0': 'To verify this claim, I need to identify the actor ...",The claim is that the actor who played Captain Hook in the live pr...,"[Taylor Louderman, Peter Pan Live!, Christopher Walken, The Deer H...",✔️ [1.000]


86.33

In [12]:
kwargs = dict(teacher_settings=dict(lm=gpt41), prompt_model=gpt41, max_errors=999)

tp = dspy.MIPROv2(metric=top5_recall, auto="medium", num_threads=16, **kwargs)
optimized_react = tp.compile(react, trainset=trainset, max_bootstrapped_demos=3, max_labeled_demos=0)

2025/05/01 14:06:40 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: True
num_candidates: 9
valset size: 80

2025/05/01 14:06:56 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/01 14:06:56 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/01 14:06:56 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=9 sets of demonstrations...


Bootstrapping set 1/9
Bootstrapping set 2/9


 40%|████      | 8/20 [01:33<02:19, 11.64s/it]


Bootstrapped 3 full traces after 8 examples for up to 1 rounds, amounting to 8 attempts.
Bootstrapping set 3/9


 15%|█▌        | 3/20 [00:27<02:36,  9.20s/it]


...

2025/05/01 14:57:28 INFO dspy.evaluate.evaluate: Average Metric: 25.333333333333332 / 35 (72.4%)
2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 72.38 on minibatch of size 35 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 2', 'Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 4'].
2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [83.81, 82.86, 85.71, 83.81, 82.86, 82.86, 84.76, 83.81, 81.9, 89.52, 81.9, 74.29, 82.86, 79.05, 85.71, 80.0, 84.76, 80.95, 86.67, 88.57, 81.9, 82.86, 86.67, 72.38]
2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [85.0, 80.42, 86.67, 86.25, 85.42]
2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.67
2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/01 14:57:28 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 30 / 31 - Minibatch ==



Average Metric: 28.67 / 35 (81.9%): 100%|██████████| 35/35 [01:46<00:00,  3.06s/it]

2025/05/01 14:59:15 INFO dspy.evaluate.evaluate: Average Metric: 28.666666666666668 / 35 (81.9%)
2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 81.9 on minibatch of size 35 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 4', 'Predictor 1: Instruction 8', 'Predictor 1: Few-Shot Set 4'].
2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [83.81, 82.86, 85.71, 83.81, 82.86, 82.86, 84.76, 83.81, 81.9, 89.52, 81.9, 74.29, 82.86, 79.05, 85.71, 80.0, 84.76, 80.95, 86.67, 88.57, 81.9, 82.86, 86.67, 72.38, 81.9]
2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [85.0, 80.42, 86.67, 86.25, 85.42]
2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.67
2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2025/05/01 14:59:15 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 31 / 31 - Full Eva


Average Metric: 66.33 / 80 (82.9%): 100%|██████████| 80/80 [01:46<00:00,  1.33s/it]

2025/05/01 15:01:02 INFO dspy.evaluate.evaluate: Average Metric: 66.33333333333333 / 80 (82.9%)
2025/05/01 15:01:02 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [85.0, 80.42, 86.67, 86.25, 85.42, 82.92]
2025/05/01 15:01:02 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.67
2025/05/01 15:01:02 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/05/01 15:01:02 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/01 15:01:02 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 86.67!


In [13]:
evaluate(optimized_react)

Average Metric: 87.67 / 100 (87.7%): 100%|██████████| 100/100 [04:15<00:00,  2.55s/it]

2025/05/01 15:07:07 INFO dspy.evaluate.evaluate: Average Metric: 87.66666666666667 / 100 (87.7%)


,claim,example_titles,trajectory,reasoning,pred_titles,top5_recall
0,The Church of England's movement that inspired the Trinity Episcop...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",{'thought_0': 'The claim involves several specific elements: a Chu...,The claim is that the Church of England's movement inspiring the T...,"[Trinity Episcopal Church (Houghton, Michigan), Oxford Movement, S...",✔️ [1.000]
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]","{'thought_0': 'The claim mentions ""Red, White & Crüe"" and a French...","The claim asserts that ""Red, White & Crüe and this athlete both fi...","[Red, White & Crüe, Bobby Stewart, Jean Baptiste Mendy, Mathieu Ba...",✔️ [0.333]
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Ed Wood, Glen or Glenda, Fernand Rivers]","{'thought_0': 'To verify the claim, I need to identify the writer/...","The claim is that the writer/director/actor from ""Glen or Glenda"" ...","[Ed Wood, Fernand Rivers, Glen or Glenda]",✔️ [1.000]
3,The film by Sandi Sissel was released before The End of Suburbia.,"[The End of Suburbia, Sandi Sissel, Chicken Ranch (film)]","{'thought_0': 'To verify the claim, I need to find out the release...",The claim states that the film by Sandi Sissel was released before...,"[Chicken Ranch (film), The End of Suburbia]",✔️ [0.667]
4,The actor who played captain hook in the live production with Tayl...,"[Christopher Walken, Peter Pan Live!, Taylor Louderman]","{'thought_0': 'To verify the claim, I need to identify the actor w...",The claim states that the actor who played Captain Hook in the liv...,"[Taylor Louderman, Peter Pan Live!, Christopher Walken, The Deer H...",✔️ [1.000]


87.67

In [14]:
optimized_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

['Up Against It', 'Joe Orton', 'Bernard-Marie Koltès']

In [15]:
dspy.inspect_history(n=2)





[2025-05-01T15:07:35.855733]

System message:

Your input fields are:
1. `claim` (str)
2. `trajectory` (str)
Your output fields are:
1. `next_thought` (str)
2. `next_tool_name` (Literal['search_wikipedia', 'lookup_wikipedia', 'finish'])
3. `next_tool_args` (dict[str, Any])
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claim ## ]]
{claim}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: search_wikipedia; lookup_wikipedia; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given a claim and the current trajectory (which includes your previous thoughts, tool selections, arg

In [ ]:
optimized_react.save("optimized_react.json")

loaded_react = dspy.ReAct("claim -> titles: list[str]", tools=[search_wikipedia, lookup_wikipedia], max_iters=20)
loaded_react.load("optimized_react.json")

loaded_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles